# Import des libs 

pour lancer le code : ```python3 main.py``` <br>
pour compiler la lib c++ : ```python3 setup.py build_ext --inplace```


In [1]:
import ctypes
import platform
from collections import defaultdict
import numpy as np
import random
import sys, os

In [2]:
if platform.system().lower() == "windows":
    lib_path = "./libs/secret_envs.dll"
elif platform.system().lower() == "linux":
    lib_path = "./libs/libsecret_envs.so"
elif platform.system().lower() == "darwin":
    if "intel" in platform.processor().lower():
        lib_path = "./libs/libsecret_envs_intel_macos.dylib"
    else:
        lib_path = "./libs/libsecret_envs.dylib"

In [3]:

# Now you can import your modules
from secret_envs_wrapper import SecretEnv0Wrapper
from secret_envs_wrapper import SecretEnv1Wrapper
from secret_envs_wrapper import SecretEnv0
from secret_envs_wrapper import SecretEnv1

try:
    import lib
    print("Library found, SUCCESS!")
except ImportError:
    print("Library not found")
    sys.exit(1)


Library found, SUCCESS!


# Monte carlo
* ES
* on policy monte carlo control
* off policy monte carlo control

In [4]:

env = SecretEnv0()
# Supposons que `env` soit votre environnement implémenté en Python
monte_carlo_es = lib.MonteCarloES(env, 10000)
value_table = monte_carlo_es.run()
print(value_table)

{6546: -36.1593002309025, 5954: -10.639699627688229, 6089: -23.85070807734307, 6214: -22.071422300346537, 5016: -26.29789818236665, 5401: -5.20255675127297, 6553: -26.359007927953222, 7960: -2.0, 6857: -12.53189249911955, 5570: -11.09821501591303, 6728: -10.466174574128356, 6784: -9.561792499119552, 6917: -8.64827525163591, 5446: -21.02672659295712, 5577: -22.764242449250432, 7832: -7.96, 6593: -22.10042120217738, 6726: -28.13563593532089, 6985: -34.59310100654364, 7369: -12.7039701198, 7496: -9.80199002, 7552: -7.8807979999999995, 8006: 0.0, 3986: -37.380247235467905, 7622: 1.9635993333333335, 8001: -3.2, 7497: -6.874527511666666, 7624: -12.484530333333332, 6337: -22.109880893213813, 6854: -27.993295674904015, 5574: -11.09821501591303, 6680: -21.544467375727937, 7465: -8.821791017999999, 6809: -24.432224973574737, 6424: -32.66106053360854, 3608: -18.122169122770917, 4114: -34.57068490289834, 3730: 12.140582955154532, 3865: 15.294757893472445, 3782: 10.325833382845339, 3606: -10.907509

: 

In [5]:
onpolicy_mc_control = lib.onPolicyMonteCarloControl(env, 10)
q_table = onpolicy_mc_control.run()
print(q_table)

: 

: 

In [ ]:
mcoffpolicy = lib.OffPolicyMonteCarloControl(env, 10000)
value_table = mcoffpolicy.run()
print(value_table)

{7638: {1: -19.701995282301905}, 7590: {1: -3.940399056460381}, 7378: {1: -9.713970388821533}, 7289: {2: -24.203761583565417}, 7625: {1: -19.701995282301905}, 7266: {1: -12.160397632724532}, 7696: {2: -2.9701000284194947}, 8001: {0: -4.0}, 7641: {1: -27.582793395222666, 2: -14.973516414549447}, 7873: {2: -3.9800000190734863}, 7506: {0: -33.306965724320456, 1: -8.801990206948702}, 7634: {0: -17.075062577994984, 1: -13.254069553548554}, 7593: {1: -7.092718301628686}, 7864: {1: -6.53239133565322, 2: -5.97000002861023}, 7896: {1: -4.643333355585734, 2: -7.8273333708445225}, 7769: {1: -12.870433456484477, 2: -7.637400073078701}, 7649: {0: -5.99117213818315, 1: -9.006626414766584, 2: -10.215849405638025}, 7522: {1: -4.900995103474351, 0: -14.217819461433452}, 7728: {2: -8.910300085258484, 1: -10.106054642154646}, 7793: {1: -9.636324536649916, 0: -12.28638010799408, 2: -9.821130760640457}, 8050: {1: -3.7722567287784656, 0: -3.861111111111112}, 7905: {1: -7.265813988308574, 0: -5.5652941395254

In [ ]:
q_learning = lib.QLearning(env, 10)
q_table = q_learning.run()
print(q_table)

# Policy iteration, politique et value function

In [ ]:
policy_iteration = lib.PolicyIteration(SecretEnv0)
policy1, V1 = policy_iteration.run()

print("Optimal policy for SecretEnv1:")
print(policy1)
print("Value function for SecretEnv1:")
print(V1)

Delta: 0
Delta: 0
Delta: 0
Delta: 0
Delta: 0
Delta: 0
Delta: 0
Delta: 0
Delta: 0
Delta: 0
Delta: 0
Delta: 0
Delta: 0
Delta: 0
Delta: 0
Delta: 0
Delta: 0
Delta: 0
Delta: 0
Delta: 0
Delta: 0
Delta: 0
Delta: 0
Delta: 0
Delta: 0
Delta: 0
Delta: 0
Delta: 0
Delta: 0
Delta: 0
Delta: 0
Delta: 0
Delta: 0
Delta: 0
Delta: 0
Delta: 0
Delta: 0
Delta: 0
Delta: 0
Delta: 0
Delta: 0
Delta: 0
Delta: 0
Delta: 0
Delta: 0
Delta: 0
Delta: 0
Delta: 0
Delta: 0
Delta: 0
Delta: 0
Delta: 0
Delta: 0
Delta: 0
Delta: 0
Delta: 0
Delta: 0
Delta: 0
Delta: 0
Delta: 0
Delta: 0
Delta: 0
Delta: 0
Delta: 0
Delta: 1
Delta: 1
Delta: 1
Delta: 1
Delta: 1
Delta: 1
Delta: 1
Delta: 1
Delta: 1
Delta: 1
Delta: 1
Delta: 1
Delta: 1
Delta: 1
Delta: 1
Delta: 1
Delta: 1
Delta: 1
Delta: 1
Delta: 1
Delta: 1
Delta: 1
Delta: 1
Delta: 1
Delta: 1
Delta: 1
Delta: 1
Delta: 1
Delta: 1
Delta: 1
Delta: 1
Delta: 1
Delta: 1
Delta: 1
Delta: 1
Delta: 1
Delta: 1
Delta: 1
Delta: 1
Delta: 1
Delta: 1
Delta: 1
Delta: 1
Delta: 1
Delta: 1
Delta: 1
Delta: 1
D

KeyboardInterrupt: 